# Collecte de tweets en utilisant Tweepy

> Ce notebook permet de récupérer des données en live depuis le site [Twitter](https://twitter.com/) en utilisant la librairie [Tweepy](https://www.tweepy.org/).

In [34]:
# import des librairies

from tweepy.streaming import StreamListener, Stream
from tweepy import OAuthHandler
from pprint import pprint
import json

## Définition des credentials

> Pour utiliser l'API Twitter, il faut obtenir des credentials. Ces credentials sont disponibles en se rendant dans [Twitter](https://developer.twitter.com/) à condition d'avoir un compte avec un numéro de téléphone et d'en faire la demande. Il faut ensuite mettre en place l'authentification OAuth V2.
>
> On récupère alors 4 clefs:
> 
> - API Key
> - API Key Secret
> - Access Token
> - Access Token Secret
> 
> 
> Je les ai placés dans un fichier `json`: 
> 
> ```json
{
    "api-key": "XXXXXXXXXXX",
    "api-key-secret": "XXXXXXXXXXXXXXXXXX",
    "access-token": "XXXXXXXXXXXXXXXX",
    "access-token-secret": "XXXXXXXXXXXXXXXXXXX"
}
```

In [62]:
# définition du chemin vers le fichier contenant les credentials de Twitter

# A REMPLACER PAR LE CHEMIN VERS VOTRE FICHIER

path_to_twitter_credentials = '/home/paul/Credentials/twitter_credentials.auth'

In [63]:
# récupération des credentials

with open(path_to_twitter_credentials, 'r') as file:
    credentials = json.load(file)
    

## Lecture des données

> Pour lire les données, on peut utiliser la classe StreamListener. Cette classe contient deux méthodes: 
>
> - `on_data(self, data)`: qui indique ce que l'on doit faire lorsqu'un tweet est reçu
> - `on_error(self, status)`: qui indique ce que l'on doit faire lorsqu'une erreur est reçue


In [64]:
# Définition de la classe MyListener

class MyListener(StreamListener):
    """
    Custom Listener of streaming Tweets that will print the data into the terminal
    """

    def __init__(self):
        """
        creates a Custom Listener of Tweets that will end after a given amount of streamed Tweets
        :param count: number of Tweets to stream
        """
        # instantiating the super class StreamListener
        StreamListener.__init__(self)
 

    def on_data(self, data):
        """
        prints name of the author of the Tweet and content in the terminal
        :param data: full data of the Tweet
        :return: True if there are still Tweets to stream, else False ending the stream
        """
        
        return True

    def on_error(self, status):
        """
        ends the stream and prints the error code
        :param status: error code
        :return: False ending the stream
        """
        print('The stream ended with status error:' + str(status))
        return False

> Dans notre cas, nous allons créer une classe `StdOutListener` qui imprime le contenu des tweets

In [65]:
class StdOutListener(StreamListener):
    """
    Custom Listener of streaming Tweets that will print the data into the terminal
    """

    def __init__(self, limit=10):
        """
        creates a Custom Listener of Tweets that will end after a given amount of streamed Tweets
        :param count: number of Tweets to stream
        """
        # instantiating the super class StreamListener
        StreamListener.__init__(self)
        self.limit = limit
        self.counter = 0
 

    def on_data(self, data):
        """
        prints name of the author of the Tweet and content in the terminal
        :param data: full data of the Tweet
        :return: True if there are still Tweets to stream, else False ending the stream
        """
        self.counter += 1 
        
        # si le counter dépasse notre limite alors le stream doit s'arrêter.
        # pour faire cela, on retourne False
        
        if self.counter > self.limit:
            return False
        data = json.loads(data)
        
        
        print(self.counter, '/', self.limit)
        print(data['user']['screen_name'])
        print(data['text'])
        print(data['created_at'])
        print()
        
        return True

    def on_error(self, status):
        """
        ends the stream and prints the error code
        :param status: error code
        :return: False ending the stream
        """
        print('The stream ended with status error:' + str(status))
        return False

## Authentification

> Le processus d'authentification est complexe:

In [66]:
# définition d'un objet OAuthHandler
authentication_handler = OAuthHandler(
    consumer_key=credentials['consumer-key'],
    consumer_secret=credentials['consumer-secret']
)

# définition de l'access token
authentication_handler.set_access_token(
    key=credentials['access-token'],
    secret=credentials['access-token-secret']
)


## Stream

> Pour lancer du streaming de données, on va pouvoir instancier notre classe. On pourra alors instancier la classe `Stream` en lui fournissant notre `Listener` ainsi que notre objet `OAuthHandler`.


In [67]:
# instanciation de la classe StdOutListener
stdout_listener = StdOutListener(limit=20)

# Instanciation du Stream
stream = Stream(
    auth=authentication_handler,
    listener=stdout_listener
)

> Ne nous reste alors plus qu'à récupérer les données en spécifiant différents mot-clefs que l'on souhaite récupérer.

In [68]:
# définition de la liste des sujets que l'on veut streamer
sujets = ['football']

# lancement du stream
stream.filter(track=sujets)

1 / 20
sibelian62
RT @BBCPhilharmonic: ⚽ What pieces of classical music should #football clubs play to inspire their players to victory?

@MusicMagazine ask…
Fri Jan 28 17:27:05 +0000 2022

2 / 20
juninho_raf
Ça me parait pas trop degueu pour remplacer Bruno
Fri Jan 28 17:27:07 +0000 2022

3 / 20
UnitedLeeds_
RT @RepTracker: Darragh Lenihan is out of contract at the end of the season 

He is eager to play PL football and is drawing interest from…
Fri Jan 28 17:27:07 +0000 2022

4 / 20
CoachBJEdwards
RT @LaRonMoore: @BrunswickFB C/o 2024 aka “THE JANE MACON 19”, This was a special moment last night at the football Banquet….This pic Spoke…
Fri Jan 28 17:27:07 +0000 2022

5 / 20
NickyDerwin
RT @james_corbett: A curious case from the world of football sponsorship.

Another company, seemingly with no staff, no active products, ba…
Fri Jan 28 17:27:07 +0000 2022

6 / 20
latestly
Cristiano Ronaldo Collects Globe Soccer’s Top Scorer of All Time Award at Expo 2020 Dubai, Interacts With Fans (Vi

## Stockage des données

> On peut aussi créer un Listener qui mettra les données dans un fichier

In [49]:
import tqdm

class RecordListener(StreamListener):
    """
    Custom Listener of streaming Tweets that will print the data into the terminal
    """

    def __init__(self, limit=10, file="raw_data/dump.jsonl"):
        """
        creates a Custom Listener of Tweets that will end after a given amount of streamed Tweets
        :param count: number of Tweets to stream
        """
        # instantiating the super class StreamListener
        StreamListener.__init__(self)
        self.limit = limit
        self.counter = 0
        self.file = file
        self.progress_bar = None
 

    def on_data(self, data):
        """
        prints name of the author of the Tweet and content in the terminal
        :param data: full data of the Tweet
        :return: True if there are still Tweets to stream, else False ending the stream
        """

        if not self.progress_bar:
            self.progress_bar = tqdm.tqdm_notebook(total=self.limit)
            
        
        
        self.counter += 1 
        

        
        # si le counter dépasse notre limite alors le stream doit s'arrêter.
        # pour faire cela, on retourne False
        
        if self.counter > self.limit:
            return False
        
        with open(self.file, 'a') as file:
            file.write(data)
            
        self.progress_bar.update(1)
        
        return True

    def on_error(self, status):
        """
        ends the stream and prints the error code
        :param status: error code
        :return: False ending the stream
        """
        print('The stream ended with status error:' + str(status))
        return False

In [69]:
# instanciation de la classe StdOutListener
stdout_listener = RecordListener(limit=10, file='raw_data/test_file.jsonl')

# Instanciation du Stream
stream = Stream(
    auth=authentication_handler,
    listener=stdout_listener
)

> Ne nous reste alors plus qu'à récupérer les données en spécifiant différents mot-clefs que l'on souhaite récupérer.

In [70]:
sujets = ['nft']

# lancement du stream
stream.filter(track=sujets)

<ipython-input-49-d4c43ac4949b>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  self.progress_bar = tqdm.tqdm_notebook(total=self.limit)


  0%|          | 0/10 [00:00<?, ?it/s]

In [71]:
# Code pour ouvrir un fichier jsonl

with open('test_file.jsonl', 'r') as file:
    lines = list(file)
    
data = [json.loads(l) for l in lines]

import pandas as pd

df = pd.DataFrame(data)
df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_tweet,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,Thu Jan 27 21:32:02 +0000 2022,1486814280943079426,1486814280943079426,"RT @EnModeMacron: Après Sarkozy, c'est au tour...","<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,None,NaN,None,...,fr,1643319122907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu Jan 27 21:32:03 +0000 2022,1486814281517735936,1486814281517735936,@Neptoon777 Et Melenchon a clairement perdu le...,"<a href=""http://twitter.com/download/iphone"" r...",True,1.486813e+18,1486813202893742083,1.247786e+18,1247786391674556419,...,fr,1643319123044,"[12, 140]",{'full_text': '@Neptoon777 Et Melenchon a clai...,NaN,NaN,NaN,NaN,NaN,NaN
2,Thu Jan 27 21:32:03 +0000 2022,1486814283472293895,1486814283472293895,"RT @avec_marine: 📊 En hausse, Marine Le Pen at...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,fr,1643319123510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu Jan 27 21:32:03 +0000 2022,1486814284852174852,1486814284852174852,RT @Very_Bad_Mo___: On est à 73 jours de l'éle...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,fr,1643319123839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thu Jan 27 21:32:04 +0000 2022,1486814289151287298,1486814289151287298,RT @PChaibriant: @FrancoiDucrocq j'ai toujours...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,fr,1643319124864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# Code pour enregister un grand nombre de tweets

sujets = ['baseball', 'football']
nb_tweets = 10
file = 'raw_data/football.jsonl'


# instanciation de la classe StdOutListener
stdout_listener = RecordListener(
    limit=nb_tweets, 
    file=file)

# Instanciation du Stream
stream = Stream(
    auth=authentication_handler,
    listener=stdout_listener
)

# lancement du stream
stream.filter(track=sujets)

<ipython-input-49-d4c43ac4949b>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  self.progress_bar = tqdm.tqdm_notebook(total=self.limit)


  0%|          | 0/10 [00:00<?, ?it/s]

## Autre utilisation de l'API

> Pour référence, le lien vers le compte de [@EmmanuelMacron](https://twitter.com/EmmanuelMacron)
> - retrouver des utilisateurs
> - retrouver des tweets, ...


In [74]:
from tweepy import API


api = API(auth_handler=authentication_handler)

# récupération d'un utilisateur
u = api.get_user('EmmanuelMacron')

In [75]:
# Donées sous forme de json
u._json

{'id': 1976143068,
 'id_str': '1976143068',
 'name': 'Emmanuel Macron',
 'screen_name': 'EmmanuelMacron',
 'location': 'France',
 'profile_location': None,
 'description': 'Président de la République française.',
 'url': 'https://t.co/tXWVLyHyWM',
 'entities': {'url': {'urls': [{'url': 'https://t.co/tXWVLyHyWM',
     'expanded_url': 'http://www.elysee.fr',
     'display_url': 'elysee.fr',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 7575878,
 'friends_count': 724,
 'listed_count': 12240,
 'created_at': 'Sun Oct 20 19:35:28 +0000 2013',
 'favourites_count': 209,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': True,
 'statuses_count': 11005,
 'lang': None,
 'status': {'created_at': 'Fri Jan 28 11:58:58 +0000 2022',
  'id': 1487032449594974208,
  'id_str': '1487032449594974208',
  'text': "À tous les agents du ministère de la Santé et des Solidarités, à ceux du centre de crise que j'ai vus à nouveau hie… h

In [78]:
# Nom dans le @
print(u.screen_name)

# Nom affiché
print(u.name)

# Description
print(u.description)

# Date de création
print(u.created_at)

# Nombre de followers
print(u.followers_count)

# Nomre d'abonnements
print(u.friends_count)

# Nombre de tweets
print(u.statuses_count)

# Dernier tweet
print(u.status)

# Dernier tweet au format JSON
pprint(u._json)

EmmanuelMacron
Emmanuel Macron
Président de la République française.
2013-10-20 19:35:28
7575878
724
11005
Status(_api=<tweepy.api.API object at 0x7f92477537f0>, _json={'created_at': 'Fri Jan 28 11:58:58 +0000 2022', 'id': 1487032449594974208, 'id_str': '1487032449594974208', 'text': "À tous les agents du ministère de la Santé et des Solidarités, à ceux du centre de crise que j'ai vus à nouveau hie… https://t.co/CzMe2TRzTU", 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/CzMe2TRzTU', 'expanded_url': 'https://twitter.com/i/web/status/1487032449594974208', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'geo': None, 'coordinates': None, 'place':

In [79]:
# Suivre un compte
u.follow()
# Arreter de suivre le compte
u.unfollow()

In [80]:
# Exemple d'un utilisateur suspendu

try:
    api.get_user('realdonaldtrump')
except Exception as e:
    print(str(e))

[{'code': 63, 'message': 'User has been suspended.'}]


> Liens vers la documentation:
>
> - de l'objet [User](https://developer.twitter.com/en/docs/twitter-api/premium/data-dictionary/object-model/user)
> - de l'objet [Status](https://developer.twitter.com/en/docs/twitter-api/premium/data-dictionary/object-model/tweet)